# Using a sequential Monte Carlo model to localize sensors based on radio pings: process ping data

## Load the libraries we need

Load the third-party libraries.

In [1]:
import numpy as np
import pandas as pd
import os

Load our `smclocalize` module.

In [2]:
from smclocalize import *

## Load the ping data

The ping data is in a set of JSON files (one for each time window) in the format produced by [tools/get_radio_obs.py](https://github.com/WildflowerSchools/sensei/blob/master/tools/get_radio_obs.py) from the [sensei repository](https://github.com/WildflowerSchools/sensei).

In [3]:
data_json_path = './data/json/'

In [4]:
json_input_files = [x for x in os.listdir(data_json_path) if x.endswith('.json')]

In [5]:
dataframes = []

In [6]:
for json_input_file in json_input_files:
    with open(os.path.join(data_json_path, json_input_file), 'r') as input_fullpath:
        dataframes.append(pd.read_json(input_fullpath))

In [7]:
all_data = pd.concat(dataframes, ignore_index = True)

In [8]:
all_data

,local_id,local_type,observed_at,remote_id,remote_type,rssi
0,8,area,2017-11-28 14:00:00,11061,child,-88
1,8,area,2017-11-28 14:00:00,11060,child,-91
2,12,area,2017-11-28 14:00:00,11060,child,-83
3,13,area,2017-11-28 14:00:00,11067,child,-86
4,9,area,2017-11-28 14:00:00,11067,child,-82
5,9,area,2017-11-28 14:00:00,11060,child,-85
6,9,area,2017-11-28 14:00:00,11061,child,-83
7,11060,child,2017-11-28 14:00:00,11061,child,-71
8,11060,child,2017-11-28 14:00:00,11067,child,-69
9,11062,child,2017-11-28 14:00:00,11067,child,-81


## Filter the ping data

Currently, the SMC model can't distinguish between a sensor which is offline and a sensor which is simply very far away from all of the other sensors. Therefore, if our ping data contains sensors that are offline, it will heavily skew our position inferences. We need to make the model more robust, but for now, we filter out sensors that are offline before attempting to perform localization.

Calculate a set of summary statistics for the pings sent by each sensor.

In [9]:
sensor_diagnostics = all_data.groupby('remote_id')['observed_at'].agg(['nunique', 'min', 'max']).rename(
    columns = {'nunique': 'n_times', 'min': 'min_time', 'max': 'max_time'}
)

Sort by the total number of time windows in which each sensor sent pings.

In [10]:
sensor_diagnostics.sort_values('n_times')

,n_times,min_time,max_time
remote_id,,,
11062,124,2017-11-28 14:00:00,2017-11-28 14:24:20
62461,151,2017-11-28 19:29:30,2017-11-28 20:00:00
47423,153,2017-11-28 14:13:30,2017-11-28 16:44:10
8,384,2017-11-28 14:00:10,2017-11-28 19:58:40
41582,610,2017-11-28 14:00:00,2017-11-28 16:16:20
13,873,2017-11-28 14:00:10,2017-11-28 17:49:40
1208468,919,2017-11-28 14:00:00,2017-11-28 17:18:40
5613,937,2017-11-28 14:00:10,2017-11-28 20:00:00
62459,951,2017-11-28 14:00:00,2017-11-28 17:08:30


A handful of sensors sent pings in very few time windows (e.g., fewer than 200 time windows out of over 2,000 time windows). We want to remove these sensors from the data set.

Sort by the earliest time that each sensor sent pings.

In [11]:
sensor_diagnostics.sort_values('min_time')

,n_times,min_time,max_time
remote_id,,,
11067,1616,2017-11-28 14:00:00,2017-11-28 20:00:00
1208468,919,2017-11-28 14:00:00,2017-11-28 17:18:40
1208464,1701,2017-11-28 14:00:00,2017-11-28 20:00:00
62459,951,2017-11-28 14:00:00,2017-11-28 17:08:30
47422,1446,2017-11-28 14:00:00,2017-11-28 18:56:30
41582,610,2017-11-28 14:00:00,2017-11-28 16:16:20
39282,1079,2017-11-28 14:00:00,2017-11-28 17:38:10
11066,1556,2017-11-28 14:00:00,2017-11-28 19:24:40
11065,1111,2017-11-28 14:00:00,2017-11-28 17:52:00


A couple of the sensors didn't send any pings for the first hour and a half or so. Similarly, we want to remove these sensors from the data set.

Sort by the latest time that each sensor sent pings.

In [12]:
sensor_diagnostics.sort_values('max_time')

,n_times,min_time,max_time
remote_id,,,
11062,124,2017-11-28 14:00:00,2017-11-28 14:24:20
41582,610,2017-11-28 14:00:00,2017-11-28 16:16:20
47423,153,2017-11-28 14:13:30,2017-11-28 16:44:10
62459,951,2017-11-28 14:00:00,2017-11-28 17:08:30
1208468,919,2017-11-28 14:00:00,2017-11-28 17:18:40
39282,1079,2017-11-28 14:00:00,2017-11-28 17:38:10
13,873,2017-11-28 14:00:10,2017-11-28 17:49:40
11065,1111,2017-11-28 14:00:00,2017-11-28 17:52:00
47422,1446,2017-11-28 14:00:00,2017-11-28 18:56:30


One sensor stopped sending pings after 24 minutes and then a punch of sensors started to cut out after a couple of hours. We want to remove the first sensor and truncate the entire data set after 16:00 UTC.

Apply all of these filters to produce a list of the sensors we want to keep in the data.

In [13]:
usable_sensors = sensor_diagnostics[(sensor_diagnostics['n_times'] > 200) &
                                    (sensor_diagnostics['min_time'] < pd.Timestamp('2017-11-28 14:30')) &
                                    (sensor_diagnostics['max_time'] > pd.Timestamp('2017-11-28 15:50'))].index.values.tolist()

We also want to remove the sensor with entity ID 8. That sensor is in the bathroom in the basement, and its data won't make sense in our current two-dimensional model.

In [14]:
usable_sensors.remove(8)

Subtract our usable sensor list from the overall list of sensors so we can see which sensors we are removing.

In [15]:
set(sensor_diagnostics.index.values.tolist()) - set(usable_sensors)

{8, 11062, 31986, 47423, 62461}

These correspond to the problematic sensors we identified above.

Filter out these sensors and remove all data after 16:00 UTC to produce the dataset we will use for all subsequent analysis.

In [16]:
usable_data = all_data[(all_data['remote_id'].isin(usable_sensors)) &
                       (all_data['local_id'].isin(usable_sensors)) &
                       (all_data['observed_at'] <= pd.Timestamp('2017-11-28 16:00'))].reset_index(drop=True)

## Extract entity IDs

Extract the list of entity IDs in our data set corresponding to each type of sensors. These lists are the basis for our variable structure.

In [17]:
child_entity_ids = np.union1d(pd.unique(usable_data[usable_data.local_type == 'child'].local_id),
                              pd.unique(usable_data[usable_data.remote_type == 'child'].remote_id)).tolist()
material_entity_ids = np.union1d(pd.unique(usable_data[usable_data.local_type == 'material'].local_id),
                                 pd.unique(usable_data[usable_data.remote_type == 'material'].remote_id)).tolist()
teacher_entity_ids = np.union1d(pd.unique(usable_data[usable_data.local_type == 'teacher'].local_id),
                                pd.unique(usable_data[usable_data.remote_type == 'teacher'].remote_id)).tolist()
area_entity_ids = np.union1d(pd.unique(usable_data[usable_data.local_type == 'area'].local_id),
                             pd.unique(usable_data[usable_data.remote_type == 'area'].remote_id)).tolist()

## Define the variable structure for the model

Using the lists of entity IDs, define an instance of the `SensorVariableStructure` class. This class provides a whole bunch of variables and helper functions for working with the data.

In [18]:
variable_structure = SensorVariableStructure(child_entity_ids,
                                             material_entity_ids,
                                             teacher_entity_ids,
                                             area_entity_ids)

## Restructure the ping data for use in the model

Using the helper functions from the `SensorVariableStructure` class, parse the data into arrays which represent the discrete and continuous components of the $\mathbf{Y}$ variables which we will use in the model.

In the below, we parse the data separately for each time step in order to mimic the real-time use case. There is also a helper function called `sensor_data_parse_multiple_timesteps()` for parsing an entire data set containing many timesteps (not shown here).

For `y_discrete_t`, use 0 to indicate that a ping was received and 1 to indicate that a ping was not received (don't ask). For `y_continuous_t`, we convert the integer RSSI values to floats (since we're treating RSSI as a continuous variable) and we just enter a 0.0 value for RSSI if no ping was received.

In [19]:
timestamps = np.sort(usable_data['observed_at'].unique())
num_timesteps = len(timestamps)
y_discrete_t = np.ones(
    (num_timesteps, variable_structure.num_y_discrete_vars),
    dtype='int')
y_continuous_t = np.zeros(
    (num_timesteps, variable_structure.num_y_continuous_vars),
    dtype='float')
for t_index in range(num_timesteps):
    (y_discrete_t[t_index], y_continuous_t[t_index]) = variable_structure.sensor_data_parse_one_timestep(
        usable_data[usable_data['observed_at'] == timestamps[t_index]])

Apply some basic sanity checks.

In [20]:
timestamp_range = pd.date_range(usable_data['observed_at'].min(), usable_data['observed_at'].max(), freq='10S')

In [21]:
np.setdiff1d(timestamp_range, timestamps)

array(['2017-11-28T14:40:10.000000000', '2017-11-28T14:40:20.000000000',
       '2017-11-28T14:40:30.000000000'], dtype='datetime64[ns]')

There were three time windows in which no ping data was received.

Check to make sure that the post-processed data and the pre-processed data agree on the total number of pings received.

In [22]:
np.sum(y_discrete_t == 0)

40447

In [23]:
np.sum(y_continuous_t != 0.0)

40447

In [24]:
len(usable_data)

40447

## Save the ping data

In [25]:
data_numpy_path = './data/numpy/'

In [26]:
data_numpy_filename = 'ping_data'

In [27]:
np.savez(
    data_numpy_path + data_numpy_filename,
    child_entity_ids = child_entity_ids,
    material_entity_ids = material_entity_ids,
    teacher_entity_ids = teacher_entity_ids,
    area_entity_ids = area_entity_ids,
    timestamps = timestamps,
    num_timesteps = num_timesteps,
    y_discrete_t = y_discrete_t,
    y_continuous_t = y_continuous_t)

## Calculate and save the same data in matrix format

In [28]:
y_discrete_all_sensors_t = np.ones(
    (num_timesteps, variable_structure.num_sensors, variable_structure.num_sensors),
    dtype='int')
y_continuous_all_sensors_t = np.zeros(
    (num_timesteps, variable_structure.num_sensors, variable_structure.num_sensors),
    dtype='float')
for t_index in range(num_timesteps):
    (y_discrete_all_sensors_t[t_index], y_continuous_all_sensors_t[t_index]) = variable_structure.sensor_data_all_sensors_one_timestep(
        usable_data[usable_data['observed_at'] == timestamps[t_index]])

In [29]:
data_numpy_filename_all_sensors = 'ping_data_all_sensors'

In [30]:
np.savez(
    data_numpy_path + data_numpy_filename_all_sensors,
    y_discrete_all_sensors_t = y_discrete_all_sensors_t,
    y_continuous_all_sensors_t = y_continuous_all_sensors_t)